# Computation of the core quantitative observables:
 * top species
 * basal species
 * Robustness
 * Assortativity
 * Gamma in/out


**Idea:**
To produce a function for each observables, and use them to create a function which, given a fw adjacency matrix, compiles a pd.Series with all the "name":vaues of the obsservables for the given FW

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import FoodWebAnalysis as fw
import networkx as nx

In [6]:
#import experimental dataset and cast as pd.Series
dataset_names = ["FW_"+ n for n in ["004","007","008","010","016_01"]]
data = pd.Series(data=[fw.load_df(nam+".csv",to_bool=True,normalize=False) for nam in dataset_names],index=dataset_names)

#import model generators and cast as pd.Series
model_names = ["cascade", "niche", "nested"]
models = pd.Series(data=[fw.generate_cascade, fw.generate_niche, fw.generate_nested],index=model_names)

# # load the sintetic foodwebs from file (import seed Ceff and Seff and rigenerate the FW from them), also appends the experimental datasets
# SEED = pd.read_json("./generated_FW/SEED_2022-01-27 19:33:27.548880.json")
# CEFF = pd.read_json("./generated_FW/CEFF_2022-01-27 19:33:27.548880.json")
# SEFF = pd.read_json("./generated_FW/SEFF_2022-01-27 19:33:27.548880.json")
# DATA = fw.generate_FW_DATA(data,models,SEED=SEED,CEFF=CEFF,SEFF=SEFF)

In [31]:
DATA = pd.read_csv("DATA.csv", index_col=0)
# DATA.drop(DATA.columns[0], axis = 1, inplace = True)

In [36]:
DATA.iloc[1,2]

'    0   1   2   3   4   5   6   7   8   9   ...  38  39  40  41  42  43  44  \\\n0    0   1   1   0   0   1   0   0   0   0  ...   0   0   1   0   0   0   0   \n1    0   0   0   0   1   0   0   0   0   1  ...   0   0   0   0   0   0   0   \n2    0   0   0   0   1   0   1   0   0   0  ...   0   0   0   0   0   1   0   \n3    0   1   1   0   0   1   0   0   0   0  ...   0   0   0   0   0   0   0   \n4    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   1   0   \n5    0   0   0   0   1   0   1   0   0   0  ...   0   0   0   0   0   1   0   \n6    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   1   0   \n7    0   0   0   0   0   0   0   0   0   0  ...   1   0   0   0   0   0   0   \n8    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   1   0   \n9    0   0   0   0   1   0   0   0   0   0  ...   0   0   0   0   0   0   0   \n10   1   0   0   1   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   \n11   0   0   0   0   1   0   1   0   0

## Top and basal species

In [ ]:
def relative_basal_species(A):
    return np.round(fw.basal_species(A).sum()/len(A),decimals=3)
def relative_top_species(A):
    return np.round(fw.top_species_cann(A).sum()/len(A),decimals=3)

In [ ]:
DATA.applymap(relative_basal_species)

,experimetal,cascade,niche,nested
FW_004,0.125,0.688,0.094,0.219
FW_007,0.125,0.812,0.312,0.271
FW_008,0.167,0.918,0.144,0.157
FW_010,0.103,0.718,0.231,0.154
FW_016_01,0.162,0.595,0.243,0.243


In [ ]:
DATA.applymap(relative_top_species)

,experimetal,cascade,niche,nested
FW_004,0.062,0.031,0.031,0.031
FW_007,0.188,0.021,0.000,0.021
FW_008,0.154,0.003,0.027,0.003
FW_010,0.077,0.026,0.000,0.026
FW_016_01,0.000,0.027,0.000,0.027


## Directed Robustness

In [ ]:
def size_giant_component(G):
    largest_cc = max(nx.connected_components(G), key=len)
    return len(largest_cc)

def directed_robustness(A,as_df=True): 
    
    if as_df:
        A = A.to_numpy()
        
    Adj = A.copy()

    sums = A.sum(axis = 0) + 2**(-32)
    M = A/sums
    E = fw.PageRank(M,c=0.7)

    size = len(E)
    removed_nodes = 0
    
    R = 1
    for i in range(size-1):
        removed = np.argmax(E)
        removed_nodes += 1

        E[removed] = 0
        
        # remove row
        Adj[removed] = 0
        # remove column
        Adj[:,removed] = 0
    
        G = nx.from_numpy_matrix(Adj, create_using=nx.Graph)
        
        if( size_giant_component(G) <= (size-removed_nodes)/2 ):
            
            R = removed_nodes/size
            return np.round(R,3)
    
    return R

In [ ]:
DATA.applymap(lambda df: directed_robustness(df,as_df=True) )

,experimetal,cascade,niche,nested
FW_004,0.844,1,1.000,0.750
FW_007,0.667,1,1.000,0.625
FW_008,0.686,1,0.955,0.874
FW_010,0.641,1,0.846,0.923
FW_016_01,1.000,1,1.000,0.838


## Assortativity

In [ ]:
# ho barato e usato networkx perche non riuscivo a velocizzare l'altra funzione a sufficienza
def make_assortativity(A): # function to map
    G = nx.DiGraph(A)
    return np.round(nx.degree_assortativity_coefficient(G, x='out', y='in'),decimals=3)

In [ ]:
DATA.applymap(make_assortativity)

,experimetal,cascade,niche,nested
FW_004,-0.384,-0.708,-0.250,0.057
FW_007,0.001,-0.565,-0.069,0.048
FW_008,-0.458,-0.472,-0.136,-0.049
FW_010,-0.489,-0.453,0.005,0.004
FW_016_01,-0.429,-0.552,-0.343,-0.098


## Gamma in/out

In [ ]:
# in order to compute gamma in and out with its errors it's useful to analyze the generated nets 
# by hand and choose the right cutoff 
indexrandom = {"cascade":1, "niche":2, "nested":3}
indexfw = 1
niche = DATA.iloc[indexfw,indexrandom["cascade"]]

In [ ]:
def random_edge_swap(A,n_swaps=None, as_df=True):
    
    monitor=False
    
    DiG = nx.DiGraph(A)         # casts A into a network x directed graph
    edgelist = list(DiG.edges)
    N = len(edgelist)
    
    if(monitor): print(f"r_edge_funct, Nedges={N}")

    if n_swaps is None:
        n_swaps = N*3

    i,j = 0,0
    while i < n_swaps and j < 10000:
        
        if(monitor and j%1000==0 and j!=0): print(j,i," swaps_target=", n_swaps)
                
        edgelist = np.array(DiG.edges)
        e1,e2 = edgelist[np.random.choice(N,2)]    # select two edges at random
        swap1 = np.array([e1[0],e2[1]])            # exchange preis between predators
        swap2 = np.array([e2[0],e1[1]])
        
        present1 = (edgelist==swap1).all(axis=1).any()  # mask to see if swap1 is alredy present
        present2 = (edgelist==swap2).all(axis=1).any()  # mask to see if swap2 is alredy present
        
        if ( not present1 ) and ( not present2 ):  # if the none of the new edges are alredy present
            DiG.remove_edge(*e1)  # remove old edges
            DiG.remove_edge(*e2)
            DiG.add_edge(*swap1)  # add swapped edges
            DiG.add_edge(*swap2)
            i += 1
        
        j+=1
            
    if as_df:
        return nx.to_pandas_adjacency(DiG)
    else:
        return nx.to_numpy_array(DiG)


def compute_assortativity(A,as_df=True):
    """Compute assortativity 
        (ATTENTION!!!!!!!A needs to be a boolean matrix)
        Returns:
                mu and computed error on mu"""
    
    if as_df:
        A = A.values.astype(bool)
    else:
        A = A.astype(bool)
        
    deg = [np.sum(a) for a in A.T if a.sum()]
    Knn = np.array([np.sum(A[mask])/(mask.sum()) for mask in A.T if mask.sum()])
    unique_deg,counts = np.unique(deg,return_counts=True)
    Knn_redux = []
    for u_deg,count in zip(unique_deg,counts):
        mask = deg==u_deg
        Knn_redux.append(Knn[mask].sum()/count)

    logu,logKnn = np.log10(unique_deg),np.log10(Knn_redux)
    params,cov = np.polyfit(logu,logKnn,1,cov=True)
    mu,q = params[0],params[1]
    
    return mu, np.sqrt(cov[0,0])

def compute_assortativity_diff(A):
    
    n0 = 15
    
    ass, Dass = compute_assortativity(A)
    ass0_arr = np.array([compute_assortativity(random_edge_swap(A))[0] for i in range(n0)])
    ass0 = ass0_arr.mean()
    Dass0 = ass0_arr.std()/np.sqrt(n0-1)
    
    #print(ass0_arr)
    #print(Dass,ass0_arr.std())
    
    ass_diff   = ass-ass0
    rDass_diff = np.sqrt(Dass**2+Dass0**2)/np.abs(ass_diff) # relative error
    return ass_diff, rDass_diff

In [ ]:
for i in range(10):
    print(compute_assortativity_diff(A))

(-0.3808257682417172, 0.4788387691861018)
(-0.3714525926219635, 0.48937812132083613)
(-0.40627057051063326, 0.4486042421541698)
(-0.39371890796399195, 0.4609172725024465)
(-0.4002011636424049, 0.45375818585077704)
(-0.3321948595212076, 0.5470373340216725)
(-0.36207505250251826, 0.5020021454637619)
(-0.37621645852946606, 0.4835222792010258)
(-0.3868626483642612, 0.4717284795804462)
(-0.3932162290118382, 0.4635942054550725)


In [ ]:
def nedges(A): # function in order to inspect the complexity of computing assortativity
    DiG = nx.DiGraph(A)         # casts A into a network x directed graph
    edgelist = list(DiG.edges)
    return len(edgelist)

print("number of edges")
DATA.applymap(nedges)

number of edges


,experimetal,cascade,niche,nested
FW_004,140,141,140,140
FW_007,221,222,220,221
FW_008,3313,3303,3294,3287
FW_010,248,248,248,249
FW_016_01,242,242,244,242


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bafb9d42-dc34-452b-a8e3-e23b911a3756' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>